# Extracción de landmarks y features con MediaPipe

Este notebook procesa los videos de `data/raw/`, extrae los landmarks con MediaPipe, calcula features cinemáticas y exporta un archivo tabular listo para modelado.

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from pathlib import Path
import json
import os
import re

# Directorio de videos y etiquetas
videos_dir = Path('../data/raw')
etiquetas_path = Path('../results/labels_por_rango.csv')

# Cargar etiquetas por rango de frames
df_labels = pd.read_csv(etiquetas_path)
df_labels.head()

,video,start_frame,end_frame,actividad
0,Video_1.mp4,18,37,parandose
1,Video_1.mp4,1,18,sentado
2,Video_1.mp4,37,40,parado
3,Video_1.mp4,40,119,caminando adelante
4,Video_1.mp4,120,155,girando


## Funciones auxiliares para extracción de features

Incluye funciones para calcular ángulos, distancias y buscar la etiqueta de actividad para cada frame.

In [7]:
def get_activity_for_frame(df_labels, video_file, frame_idx):
    """Busca la etiqueta de actividad para un frame dado de un video, permitiendo coincidencia parcial de nombre."""

    # Normalizar nombre: solo base y sin espacios
    base_name = os.path.splitext(os.path.basename(video_file))[0].replace(' ', '_').lower()

    # Buscar coincidencias parciales en la columna video
    mask = df_labels['video'].apply(lambda v: base_name in os.path.splitext(os.path.basename(str(v)))[0].replace(' ', '').lower())
    rows = df_labels[mask]
    for _, row in rows.iterrows():
        if row['start_frame'] <= frame_idx <= row['end_frame']:
            return row['actividad']
    return None

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
    return np.degrees(angle)

def calculate_distance(p1, p2):
    p1 = np.array(p1)
    p2 = np.array(p2)
    return np.linalg.norm(p1 - p2)

## Extracción de features por ventana

Procesa cada video, extrae landmarks con MediaPipe y calcula features agregadas por ventana (media, std, min, max de ángulos, distancias, inclinaciones, etc.).

In [8]:
mp_pose = mp.solutions.pose

# Parámetros de la ventana
target_fps = 30  
window_size = int(0.5 * target_fps)  # 0.5 segundos
step_size = int(0.25 * target_fps)   # 50% solapamiento

features_list = []

for video_path in videos_dir.glob('*.mp4'):
    print(f"Procesando video: {video_path}")
    cap = cv2.VideoCapture(str(video_path))
    video_file = video_path.name
    frame_landmarks = []
    frame_idx = 0
    with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5) as pose:
        while True:
            ret, frame = cap.read()
            if not ret:
                print(f"  Fin de video o error de lectura en frame {frame_idx}")
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)
            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark
                coords = {
                    'hip': (lm[23].x, lm[23].y),
                    'knee': (lm[25].x, lm[25].y),
                    'ankle': (lm[27].x, lm[27].y),
                    'shoulder': (lm[11].x, lm[11].y)
                }
                frame_landmarks.append(coords)
            else:
                frame_landmarks.append(None)
            frame_idx += 1
    cap.release()
    print(f"  Total frames procesados: {len(frame_landmarks)}")

    # Procesar por ventana
    n_frames = len(frame_landmarks)
    for start in range(0, n_frames - window_size + 1, step_size):
        window = frame_landmarks[start:start+window_size]
        # Filtrar frames válidos
        window_valid = [f for f in window if f is not None]
        print(f"    Ventana {start}-{start+window_size-1}: {len(window_valid)}/{window_size} frames válidos")
        if len(window_valid) < window_size * 0.7:
            print("      Saltando ventana por demasiados frames perdidos")
            continue  # Demasiados frames perdidos
        
        # Calcular features agregadas
        hips = np.array([f['hip'] for f in window_valid])
        knees = np.array([f['knee'] for f in window_valid])
        ankles = np.array([f['ankle'] for f in window_valid])
        shoulders = np.array([f['shoulder'] for f in window_valid])
        
        # Ángulo rodilla (media, std)
        knee_angles = [calculate_angle(hip, knee, ankle) for hip, knee, ankle in zip(hips, knees, ankles)]
        
        # Inclinación tronco (ángulo entre hombro-cadera y vertical)
        trunk_incl = [np.degrees(np.arctan2(s[0]-h[0], h[1]-s[1])) for s, h in zip(shoulders, hips)]
        
        # Distancia hombro-cadera
        dist_sh_hip = [calculate_distance(s, h) for s, h in zip(shoulders, hips)]
        
        # Etiqueta de actividad para la ventana (mayoría de frames)
        labels = [get_activity_for_frame(df_labels, video_file, f) for f in range(start, start+window_size)]
        actividad = max(set(labels), key=labels.count) if labels and any(labels) else None
        print(f"      Labels en ventana: {labels}")
        print(f"      Actividad asignada: {actividad}")
        
        # Features agregadas
        feats = {
            'video': video_file,
            'start_frame': start,
            'end_frame': start+window_size-1,
            'knee_angle_mean': np.mean(knee_angles),
            'knee_angle_std': np.std(knee_angles),
            'trunk_incl_mean': np.mean(trunk_incl),
            'trunk_incl_std': np.std(trunk_incl),
            'dist_sh_hip_mean': np.mean(dist_sh_hip),
            'dist_sh_hip_std': np.std(dist_sh_hip),
            'actividad': actividad
        }
        features_list.append(feats)

# Guardar features
if not features_list:
    print("ADVERTENCIA: features_list está vacío. No se generaron features.")
else:
    print(f"features_list contiene {len(features_list)} elementos. Ejemplo:")
    print(features_list[0]) if len(features_list) > 0 else None

# Convertir a DataFrame solo si hay datos
if features_list:
    features_df = pd.DataFrame(features_list)
    print("DataFrame creado. Columnas:", features_df.columns)
    print(features_df.head())
else:
    features_df = pd.DataFrame()
    print("DataFrame vacío.")
features_df.to_csv('../results/features.csv', index=False)

Procesando video: ..\data\raw\Video 1.mp4
  Fin de video o error de lectura en frame 409
  Total frames procesados: 409
    Ventana 0-14: 15/15 frames válidos
      Labels en ventana: [None, 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado']
      Actividad asignada: sentado
    Ventana 7-21: 15/15 frames válidos
      Labels en ventana: ['sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'sentado', 'parandose', 'parandose', 'parandose', 'parandose']
      Actividad asignada: sentado
    Ventana 14-28: 15/15 frames válidos
      Labels en ventana: ['sentado', 'sentado', 'sentado', 'sentado', 'parandose', 'parandose', 'parandose', 'parandose', 'parandose', 'parandose', 'parandose', 'parandose', 'parandose', 'parandose', 'parandose']
      Actividad asignada: parandose
    Ventana 21-35: 15/15 frames válidos
      Labels en vent